In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/01/30 12:23:23 WARN Utils: Your hostname, Sagars-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
25/01/30 12:23:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/30 12:23:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/Applications/saggydev/projects_learning/data_engineering_course/.venv/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""
SELECT 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|   7|2020-01-01 10:00:00|357.84000000000015|            28|
| 127|2020-01-01 23:00:00|             92.16|             2|
|  65|2020-01-21 16:00:00|            147.64|            10|
| 228|2020-01-09 05:00:00|235.15000000000003|            11|
|  66|2020-01-26 14:00:00|             80.83|             5|
|  75|2020-01-23 16:00:00|394.58000000000015|            25|
|  42|2020-01-07 16:00:00|            147.55|            10|
|  83|2020-01-18 17:00:00|             76.91|             6|
| 130|2020-01-17 06:00:00|200.17000000000002|            10|
|  95|2020-01-17 00:00:00|            246.16|            14|
| 152|2020-01-12 02:00:00|            136.09|             6|
|  95|2020-01-28 05:00:00|428.55000000000007|            25|
| 210|2020-01-10 14:00:00|            156.28|             6|
|  74|2020-01-30 10:00:0

In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/30 12:23:36 WARN MemoryManager: Total allocation exceeds 95.0

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

25/01/30 12:23:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
df_yellow_revenue \
    .repartition(4) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [11]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [12]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [13]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [14]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/30 12:24:09 WARN MemoryManager: Total allocation exceeds 95.0

In [15]:
df_join = spark.read.parquet('data/report/revenue/total')

In [16]:
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   1|              NULL|                NULL|              0.31|                    1|
|2020-01-01 00:00:00|  28|              NULL|                NULL| 96.39999999999999|                    3|
|2020-01-01 00:00:00|  52|             89.18|                   2|52.089999999999996|                    2|
|2020-01-01 00:00:00|  65| 78.83999999999999|                   4|            132.55|                    5|
|2020-01-01 00:00:00|  69|             99.61|                   5|              NULL|                 NULL|
|2020-01-01 00:00:00|  73|              NULL|                NULL|             31.06|                    1|
|2020-01-01 00:00:00|  80|16

In [18]:
df_zones = spark.read.parquet('zones/')

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/zones.

In [57]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [62]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [18]:
spark.stop()